In [ ]:
import mxnet as mx
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
# plt.style.available
plt.style.use('seaborn-poster')

## Generate sample data

In [ ]:
N = 100 # number of points per class
D = 2 # dimensionality
K = 3 # number of classes
X = np.zeros((N*K,D)) # data matrix (each row = single example)
y = np.zeros(N*K, dtype='uint8') # class labels
for j in xrange(K):
    ix = range(N*j,N*(j+1))
    r = np.linspace(0.0,1,N) # radius
    t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2 # theta
    X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
    y[ix] = j
    
# lets visualize the data:
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)

In [ ]:
num_samples = X.shape[0]

X.shape, y.shape

In [ ]:
# A single method to initilize the Neural Network parameters

def model_init(sym, lr, mm):
    
    model = mx.mod.Module(symbol=sym, context=mx.cpu(), data_names=['data'], label_names=['softmax_label'])

    model.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)

    # initialize parameters by uniform random numbers
    model.init_params(initializer=mx.init.Uniform(scale=.1))

    # use SGD with learning rate to train
    model.init_optimizer(optimizer='sgd', optimizer_params=({'learning_rate': lr, 'momentum': mm}))

    return model

In [ ]:
# Method to train the model

import time

def model_train(model, train_iter, metric, nepoch):
    # train n epochs, i.e. going over the data iter one pass
    for epoch in range(nepoch):
        train_iter.reset()
        metric.reset()
        
        st = time.time()
        
        for batch in train_iter:
            model.forward(batch, is_train=True)       # compute predictions
            model.update_metric(metric, batch.label)  # accumulate prediction accuracy
            model.backward()                          # compute gradients
            model.update()                            # update parameters
        
        et = time.time()
        
        print('Epoch %d, Training %s, Time %s' % (epoch, metric.get(), et - st))
        
    return model

In [ ]:
# Put data into batches for MxNet

batch_size = 1
train_iter = mx.io.NDArrayIter(X,y, batch_size, shuffle=True)

## Single Layer Neural Network - NN without hidden layers

In [ ]:
# input layer
data = mx.sym.Variable('data')

# output layer
fc2 = mx.sym.FullyConnected(data, name='fc2', num_hidden=3)
softmax = mx.sym.SoftmaxOutput(fc2, name='softmax') # use softmax to get the decision percentage

mx.viz.plot_network(softmax)

In [ ]:
model = model_init(softmax, 0.0001, 0.9)

# use accuracy as the metric
metric = mx.metric.create('acc')

# train the model
model = model_train(model, train_iter, metric, 100)

In [ ]:
# Method to visualize decision boundary of the classifier

def plot_decision_boundary(model, X, title, cnt=None):
    mod = model
    
    # create a mesh to plot in
    h = .02
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    mesh_data = np.c_[xx.ravel(), yy.ravel()]
    
    eval_iter = mx.io.NDArrayIter(mesh_data, batch_size=1)
    probs = mod.predict(eval_iter).asnumpy()
    
    Z = np.argmax(probs, axis=1)
    Z = Z.reshape(xx.shape)
    
    accuracy = mod.score(train_iter, 'acc')[0][1]
    
    plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu)
    plt.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.RdYlBu, s=40, edgecolors='k')
    
    plt.title('%s - Accuracy: %s' % (title, accuracy))
    
    if cnt != None:
        plt.savefig('sc/decision-%s.png' % cnt)
    
plot_decision_boundary(model, X, 'NN Without Hidden Layer')

## 2-Layer Neural Network with 1 neuron

In [ ]:
# input layer
data = mx.sym.Variable('data')

# hidden layer
fc1 = mx.sym.FullyConnected(data, name='fc1', num_hidden=1)
relu = mx.sym.Activation(fc1, name='relu1', act_type='relu')

# output layer
fc2 = mx.sym.FullyConnected(relu, name='fc2', num_hidden=3)
softmax = mx.sym.SoftmaxOutput(fc2, name='softmax') # use softmax to get the decision percentage

mx.viz.plot_network(softmax)

In [ ]:
model2 = model_init(softmax, 0.001, 0.9)

# use accuracy as the metric
metric = mx.metric.create('acc')

# train the model
model2 = model_train(model2, train_iter, metric, 100)

In [ ]:
plot_decision_boundary(model2, X, "NN With Hidden Layer - 1 neuron")

## 2-Layer Neural Network with 6 neurons

In [ ]:
# input layer
data = mx.sym.Variable('data')

# hidden layer
fc1 = mx.sym.FullyConnected(data, name='fc1', num_hidden=6)
relu = mx.sym.Activation(fc1, name='relu1', act_type='relu')

# output layer
fc2 = mx.sym.FullyConnected(relu, name='fc2', num_hidden=3)
softmax = mx.sym.SoftmaxOutput(fc2, name='softmax') # use softmax to get the percentage

mx.viz.plot_network(softmax)

In [ ]:
model3 = model_init(softmax, 0.001, 0.9)

# use accuracy as the metric
metric = mx.metric.create('acc')

# train the model
model3 = model_train(model3, train_iter, metric, 100)

In [ ]:
plot_decision_boundary(model3, X, "NN With Hidden Layer - 6 neuron")

## 2-Layer Neural Network with 20 neurons

In [ ]:
# input layer
data = mx.sym.Variable('data')

# hidden layer
fc1 = mx.sym.FullyConnected(data, name='fc1', num_hidden=20)
relu = mx.sym.Activation(fc1, name='relu1', act_type='relu')

# output layer
fc2 = mx.sym.FullyConnected(relu, name='fc2', num_hidden=3)
softmax = mx.sym.SoftmaxOutput(fc2, name='softmax') # use softmax to get the percentage

mx.viz.plot_network(softmax)

In [ ]:
model4 = model_init(softmax, 0.001, 0.9)

# use accuracy as the metric
metric = mx.metric.create('acc')

# train the model
model4 = model_train(model4, train_iter, metric, 100)

In [ ]:
plot_decision_boundary(model4, X, "NN With Hidden Layer - 20 neuron")

## 2-Layer Neural Network with 100 neurons

In [ ]:
# input layer
data = mx.sym.Variable('data')

# hidden layer
fc1 = mx.sym.FullyConnected(data, name='fc1', num_hidden=100)
relu = mx.sym.Activation(fc1, name='relu1', act_type='relu')

# output layer
fc2 = mx.sym.FullyConnected(relu, name='fc2', num_hidden=3)
softmax = mx.sym.SoftmaxOutput(fc2, name='softmax') # use softmax to get the percentage

mx.viz.plot_network(softmax)

In [ ]:
model5 = model_init(softmax, 0.005, 0.9)

# use accuracy as the metric
metric = mx.metric.create('acc')

# train the model
model5 = model_train(model5, train_iter, metric, 200)

In [ ]:
plot_decision_boundary(model5, X, "NN With Hidden Layer - 100 neuron")